In [19]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [20]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [21]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



In [22]:
# Some more magic so that the notebook will reload external python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

##### Define_Basic_Parameters

In [5]:
# Define the time-range
start_time = '2017-01-01'
end_time   = '2019-12-31'

# Create the year_name for naming variables
year_name = f'{start_time[:4]}_{end_time[:4]}'

# Define the interval for looping
Interval_index = [1] + list(range(0,121,10))[1:] 

In [6]:
# import some spatial constrains
North_china_plain_boundary = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain/0_1_North_China_Plain_Full")

# import verified points
Verified_Point_2017_2019   = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_2017_2019")

##### Prepare needed images

In [25]:
# import Landsat img
Landsat_img = ee.Image("users/wangjinzhulala/North_China_Plain_Python/Cloud_Free_Img/Cloud_Free_2017_2019").clip(North_china_plain_boundary)
Landsat_img_band = Landsat_img.bandNames().getInfo()

# import Fourier img   
Fourier_img = ee.Image("users/wangjinzhulala/North_China_Plain_Python/Fourier_pixels/Fourier_Range_3_Year_2017_2019").clip(North_china_plain_boundary)

# Import the Sentinel img
# Sentinel_img =  ee.Image("users/wangjinzhulala/North_China_Plain_Python/Supply_img/Sentinel_2019_mean")
# Sentinel_V_H =  ee.Image("users/wangjinzhulala/North_China_Plain_Python/Supply_img/Year_2017_2018_Sentinel_V_H")
# Sentinel_img =  Sentinel_img.select(['VV','VH']).addBands(Sentinel_V_H)

# Import the Index-mean Img
NDVI_img = ee.Image("users/wangjinzhulala/North_China_Plain_Python/Supply_img/Year_2017_2019_Spectrum_NDVI").rename('Nomalized_NDVI')
NDBI_img = ee.Image("users/wangjinzhulala/North_China_Plain_Python/Supply_img/Year_2017_2019_Spectrum_NDBI").rename('Nomalized_NDBI')
EVI_img  = ee.Image("users/wangjinzhulala/North_China_Plain_Python/Supply_img/Year_2017_2019_Spectrum_EVI").rename('Nomalized_EVI')

# put all band names together
Overlay_img      = ee.Image([Fourier_img,NDVI_img,NDBI_img,EVI_img])
Overlay_img_band = Overlay_img.bandNames().getInfo()

##### Img value extraction and export

In [27]:
Sample_extract_img = ee.Image([Landsat_img,Fourier_img]).sampleRegions(collection = Verified_Point_2017_2019,
                                                                       
                                                                       # [Built] is the colum that holds visualy 
                                                                       # interpreted information on sample points
                                                                       properties = ['Built'], 
                                                                       
                                                                       scale      = 30, 
                                                                       geometries = True)


In [31]:
# define the gee-asset path for exporting
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value'

# exporting
task = ee.batch.Export.table.toAsset(   collection  = Sample_extract_img,
                                        description = f'Exporting sample {year_name} with img values',
                                        assetId     = f'{path}/Sample_{year_name}_extract_Landsat_Fourier_Normalized_img')

task.start()